# Explore the dataset


In this notebook, we will perform an EDA (Exploratory Data Analysis) on the processed Waymo dataset (data in the `processed` folder). In the first part, you will create a function to display 

In [2]:
from utils import get_dataset
import cv2
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from PIL import Image
%matplotlib inline

ModuleNotFoundError: No module named 'object_detection'

In [ ]:
dataset = get_dataset("data/waymo/*.tfrecord")

## Write a function to display an image and the bounding boxes

Implement the `display_instances` function below. This function takes a batch as an input and display an image with its corresponding bounding boxes. The only requirement is that the classes should be color coded (eg, vehicles in red, pedestrians in blue, cyclist in green).

In [1]:
def display_instances(batch):
    """
    This function takes a batch from the dataset and display the image with 
    the associated bounding boxes.
    """
    # color for different classes
    colormap = {1: 'r', 2: 'b', 4: 'g'}
    plt.figure(figsize=(20,10))
    
    for data in batch:
        ax = plt.subplot(4, 3, i+1)
        imageShape = data["original_image_spatial_shape"].numpy()
        plt.imshow(data['image'].numpy().astype("uint8"))
    
    for cl, bb in zip(data['groundtruth_classes'].numpy(), data['groundtruth_boxes'].numpy()):
        y1, x1, y2, x2 = bb
        y1 *= imageShape[0]*640/1280
        x1 *= imageShape[1]*640/1920
        y2 *= imageShape[0]*640/1280
        x2 *= imageShape[1]*640/1920
        rec = Rectangle((x1, y1), x2 - x1, y2 - y1, facecolor='none',
                           edgecolor=colormap[cl])
        ax.add_patch(rec)
        
    plt.tight_layout()
    plt.show()

## Display 10 images 

Using the dataset created in the second cell and the function you just coded, display 10 random images with the associated bounding boxes. You can use the methods `take` and `shuffle` on the dataset.

In [ ]:
## STUDENT SOLUTION HERE

for i in range(10):
        batch = dataset.shuffle(100, reshuffle_each_iteration=True).take(1)
display_instances(batch)

## Additional EDA

In this last part, you are free to perform any additional analysis of the dataset. What else would like to know about the data?
For example, think about data distribution. So far, you have only looked at a single file...